In [15]:
import sys

from common.config import get_config

from common.generate_images_labels import get_images_labels_list
from common.generate_images_labels import write_images_labels_to_file


def main():
    images_source = 'train'
    if images_source not in ['train', 'test']:
        print("Invalid image-source '{}'!".format(images_source))
        return

    images_dir_path = get_config('{}ing_images_dir_path'.format(images_source))
    images_labels_path = get_config(
        '{}ing_images_labels_path'.format(images_source))

    print("Gathering info about images at path '{}'...".format(images_dir_path))
    images_labels_list = get_images_labels_list(images_dir_path)
    print("Done!")

    print("Writing images labels info to file at path '{}'...".format(
        images_labels_path))
    write_images_labels_to_file(images_labels_list, images_labels_path)
    print("Done!")
    
    images_source = 'test'
    if images_source not in ['train', 'test']:
        print("Invalid image-source '{}'!".format(images_source))
        return

    images_dir_path = get_config('{}ing_images_dir_path'.format(images_source))
    images_labels_path = get_config(
        '{}ing_images_labels_path'.format(images_source))

    print("Gathering info about images at path '{}'...".format(images_dir_path))
    images_labels_list = get_images_labels_list(images_dir_path)
    print("Done!")

    print("Writing images labels info to file at path '{}'...".format(
        images_labels_path))
    write_images_labels_to_file(images_labels_list, images_labels_path)
    print("Done!")


if __name__ == '__main__':
    main()

Gathering info about images at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\images\train'...
Done!
Writing images labels info to file at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\training_images_labels.txt'...
Done!
Gathering info about images at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\images\test'...
Done!
Writing images labels info to file at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\testing_images_labels.txt'...
Done!


In [16]:
"""
Takes a set of images as inputs, transforms them using multiple algorithms to
make it suitable for ingestion into ML routines, then finally outputs them
to disk.
"""
import csv
import traceback

import numpy as np
import cv2

from common.config import get_config
from common.image_transformation import apply_image_transformation


def write_frame_to_file(frame, frame_label, writer):
    """
    Convert the multi-dimensonal array of the image to a one-dimensional one
    and write it to a file, along with its label.
    """
   # print("Writing frame to file...")

    flattened_frame = frame.flatten()
    output_line = [frame_label] + np.array(flattened_frame).tolist()
    writer.writerow(output_line)

    #print("Done!")


def main():
    images_transformed_path = get_config('images_transformed_path')
    with open(images_transformed_path, 'w') as output_file:
        writer = csv.writer(output_file, delimiter=',')

        training_images_labels_path = get_config('training_images_labels_path')
        with open(training_images_labels_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            #print("\n\n" + line.strip())
            image_path, image_label = line.split()

            # Read the input image.
            frame = cv2.imread(image_path)
            # `frame` is a HxW numpy ndarray of triplets (pixels), where H and W are
            # the dimensions of the input image.
            # cv2.imshow("Original", frame)
            try:
                frame = apply_image_transformation(frame)
                write_frame_to_file(frame, image_label, writer)
            except Exception:
                exception_traceback = traceback.format_exc()
                print("Error while applying image transformation on image path '{}' with the following exception trace:\n{}".format(
                    image_path, exception_traceback))
                continue
            # cv2.waitKey(1000)
    cv2.destroyAllWindows()
    print ("The program completed successfully !!")


if __name__ == '__main__':
    main()

The program completed successfully !!


In [3]:
# """
# Takes a set of images as inputs, transforms them using multiple algorithms to
# make it suitable for ingestion into ML routines, then finally outputs them
# to disk.
# """
# import csv
# import traceback

# import numpy as np
# import cv2

# from common.config import get_config
# from common.image_transformation import apply_image_transformation


# def write_frame_to_file(frame, frame_label, writer):
#     """
#     Convert the multi-dimensonal array of the image to a one-dimensional one
#     and write it to a file, along with its label.
#     """
#    # print("Writing frame to file...")

#     flattened_frame = frame.flatten()
#     output_line = [frame_label] + np.array(flattened_frame).tolist()
#     writer.writerow(output_line)

#     #print("Done!")


# def main():
#     images_transformed_path = get_config('images_transformed_path')
#     with open(images_transformed_path, 'w') as output_file:
#         writer = csv.writer(output_file, delimiter=',')

#         testing_images_labels_path = get_config('testing_images_labels_path')
#         with open(testing_images_labels_path, 'r') as file:
#             lines = file.readlines()

#         for line in lines:
#             #print("\n\n" + line.strip())
#             image_path, image_label = line.split()

#             # Read the input image.
#             frame = cv2.imread(image_path)
#             # `frame` is a HxW numpy ndarray of triplets (pixels), where H and W are
#             # the dimensions of the input image.
#             # cv2.imshow("Original", frame)
#             try:
#                 frame = apply_image_transformation(frame)
#                 write_frame_to_file(frame, image_label, writer)
#             except Exception:
#                 exception_traceback = traceback.format_exc()
#                 print("Error while applying image transformation on image path '{}' with the following exception trace:\n{}".format(
#                     image_path, exception_traceback))
#                 continue
#             # cv2.waitKey(1000)
#     cv2.destroyAllWindows()
#     print ("The program completed successfully !!")


# if __name__ == '__main__':
#     main()

The program completed successfully !!


In [4]:
# import pandas as pd
# test_input=pd.read_csv('C:\\Users\\Subham\\Documents\\Indian-Sign-Language\\data\\generated\\images_transformed.csv',header=None)
# target=test_input[0]
# test_input=test_input.drop(0,axis=1)

In [17]:
import os
import sys
import csv
import numpy
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
import keras
from keras.layers import Dense
from keras.models import Sequential
from common.config import get_config


def print_with_precision(num):
    return "%0.5f" % num



def read_images_transformed(images_transformed_path):
    print("\nReading the transformed images file located at path '{}'...".format(
        images_transformed_path))

    images = []
    labels = []
    with open(images_transformed_path) as images_transformed_file:
        reader = csv.reader(images_transformed_file, delimiter=',')
        cnt = 0
        for line in reader:
            if len(line)==0:
                continue
            cnt = cnt + 1
            label = line[0]
            labels.append(label)
            image = line[1:]
            image_int = [int(pixel) for pixel in image]
            image = np.array(image_int)
            images.append(image)
    print("Done!\n")
    return images, labels


def generate_MLP_classifier():
    print("\nGenerating MLP model ...")
    # Save the number of columns in predictors: n_cols
    n_cols = 900

    # Set up the model: model
    model = Sequential()

    # Add the first layer
    model.add(Dense(32,activation='relu',input_shape=(n_cols,)))

    # Add the output layer
    model.add(Dense(1,activation='softmax'))

    # Compile the model
    model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
    print("Done!\n")
    return model

def generate_classifier(model_name):
    classifier_generator_function_name = "generate_{}_classifier".format(
        model_name)
    return globals()[classifier_generator_function_name]()


def divide_data_train_test(images, labels, ratio):
    print("\nDividing dataset in the ratio '{}' using `train_test_split()`:".format(ratio))
    ret = train_test_split(images, labels, test_size=ratio, random_state=0)
    print("Done!\n")
    return ret

In [18]:
model_name = 'MLP'
model_output_dir_path = get_config('model_{}_output_dir_path'.format(model_name))
print("Model output directory path: \n"+model_output_dir_path)
model_stats_file_path = os.path.join(model_output_dir_path, "stats-{}.txt".format(model_name))
print("\n\n")
print("Model stats will be written to the file at path \n'{}'.".format(model_stats_file_path))

Model output directory path: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP



Model stats will be written to the file at path 
'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\output\MLP\stats-MLP.txt'.


In [19]:
import pandas as pd
with open(model_stats_file_path, "w") as model_stats_file:
    
        images_transformed_path = get_config('images_transformed_path')
        print("Image Transformed Path:: \n"+images_transformed_path)
        
        images, labels = read_images_transformed(images_transformed_path)
        
        
        classifier_model = generate_classifier(model_name)
        model_stats_file.write("Model used = '{}'".format(model_name))
        model_stats_file.write("Classifier \n Model details:\n{}\n\n".format(classifier_model))
        print('IMAGES\n\n')
        all_images=pd.DataFrame(images)
        print('\n\n')
        print('LABELS\n\n')
        print(labels)
        training_images, testing_images, training_labels, testing_labels = divide_data_train_test(images, labels, 0.2)
        
        print("\nTraining the model...")
        classifier_model = classifier_model.fit(training_images, training_labels)
        print("Done!\n")

        model_serialized_path = get_config('model_{}_serialized_path'.format(model_name))
        print('Model Serialized Path \n')
        print(model_serialized_path)
        print('\n\n')
        print("\nDumping the trained model to disk at path '{}'...".format(model_serialized_path))
        joblib.dump(classifier_model, model_serialized_path)
        print("Dumped\n")

        print("\nWriting model stats to file...")
        score = classifier_model.score(testing_images, testing_labels)
        model_stats_file.write("Model score:\n{}\n\n".format(print_with_precision(score)))

        predicted = classifier_model.predict(testing_images)
        report = metrics.classification_report(testing_labels, predicted)
        model_stats_file.write("Classification report:\n{}\n\n".format(report))
        print("Done!\n")

        print("\nFinished!\n")

Image Transformed Path:: 
C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv

Reading the transformed images file located at path 'C:\Users\Subham\Documents\Indian-Sign-Language\data\generated\images_transformed.csv'...


Error: line contains NULL byte

In [14]:
all_images

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,255,212,42,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,166,255,255,255,255
2,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,229,25,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,242,46,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,255,255,255,123,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,64,242,255,255,255,255,255,255
9,0,0,0,0,0,0,0,0,0,0,...,255,255,255,89,0,0,0,0,0,0
